In [8]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
raw = pd.read_pickle('data/labelled_dataset.pickle')
raw.head()

,text,labelmax
0,Pros - The people who work here are brilliant ...,customer
1,Pros Start-up vibes Fast growing company Tech-...,customer
2,"Pros The team is great, I love the ambition of...",collaboration
3,"Pros The company is constantly growing, and at...",adaptability
4,Pros Cool office. Friendly people. Good atmosp...,collaboration


<91580x61904 sparse matrix of type '<class 'numpy.float64'>'
	with 4897030 stored elements in Compressed Sparse Row format>